# Challenge 4: Datashader

When plotting a million or more sources, standard plotting packages (like `matplotlib`)
are not going to work: the points overlap too much, even if point transparency is set low.

Instead, challenge yourself to use the packages `bokeh`, `holoviews`, and `datashader`
to make interactive plots for large data sets.

This challenge is completely open!! Make whatever cool plot you want and share with the class.


## Resources

ADQL Recipes. <br>
https://dp0-2.lsst.io/data-access-analysis-tools/adql-recipes.html#adql-recipes

DP0.2 Catalog Data Products. <br>
https://dp0-2.lsst.io/data-products-dp0-2/index.html#catalogs

Tutorial notebook 02, Catalog Queries with TAP.<br>
Tutorial notebook 06b, Interactive Catalog Visualization. <br>
Find them in your `notebooks/tutorial-notebooks/` directory.


## Hints

For this challenge you get more of a template than just a hint.

Here's the process to recreate the color-magnitude diagram for a million point-like
sources from Section 5 of tutorial notebook 06b.

Import packages.

In [ ]:
from lsst.rsp import get_tap_service
import datashader as dsh
from bokeh.io import output_notebook
import holoviews as hv
from holoviews import streams
from holoviews.operation.datashader import dynspread, datashade

Set parameters to show plots in the notebook.

In [ ]:
hv.extension('bokeh')
output_notebook()

Instantiate the TAP service.

In [ ]:
service = get_tap_service("tap")

Query for and return data for at least 1 million Objects (takes ~1 minute)
around an arbitrary sky coordinate.

In [ ]:
query = "SELECT coord_ra, coord_dec, objectId, r_extendedness, "\
        "scisql_nanojanskyToAbMag(g_cModelFlux) AS mag_g_cModel, "\
        "scisql_nanojanskyToAbMag(r_cModelFlux) AS mag_r_cModel, "\
        "scisql_nanojanskyToAbMag(i_cModelFlux) AS mag_i_cModel "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec),"\
        "CIRCLE('ICRS', 55.65, -40, 1)) = 1 " \
        "AND detect_isPrimary = 1 "\
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) < 27.0 "\
        "AND r_extendedness IS NOT NULL"
data = service.search(query).to_table().to_pandas()
print(len(data))

Compute _g-r_ and _r-i_ colors for all objects.

In [ ]:
data['gmr'] = data['mag_g_cModel'] - data['mag_r_cModel']
data['rmi'] = data['mag_r_cModel'] - data['mag_i_cModel']

Define the plot's points and bounding box.

In [ ]:
points = hv.Points((data['gmr'], data['rmi']))
boundsxy = (0, 0, 0, 0)
box = streams.BoundsXY(source=points, bounds=boundsxy)
bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

Define the plot itself, and display it.

In [ ]:
p = dynspread(datashade(points, cmap="Viridis"))
p = p.opts(width=800, height=300, padding=0.05, show_grid=True,
           xlim=(-2.0, 7.0), ylim=(-5.0, 3.0), xlabel="(g-r)", ylabel="(r-i)")
p * bounds

Use the icon with the dashed-box and magnifying glass to zoom in, and notice how the plot automatically adjusts from heatmap to individual points, allowing the million points to be visualized all together.

Use the two-arrows-in-a-circle icon to reset the plot.

**NOW MAKE YOUR OWN, BELOW!**

In [ ]:
del query, data
del points, boundsxy, box, bounds, p

# My solution is FIRE

It might not be scientifically informative but it is FUN.

In [ ]:
import numpy as np

In [ ]:
query = "SELECT coord_ra, coord_dec, objectId, r_extendedness, "\
        "scisql_nanojanskyToAbMag(r_cModelFlux) AS mag_r_cModel, "\
        "scisql_nanojanskyToAbMag(i_cModelFlux) AS mag_i_cModel "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec),"\
        "CIRCLE('ICRS', 55.65, -40, 2)) = 1 " \
        "AND detect_isPrimary = 1 "\
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) < 27.0 "\
        "AND r_extendedness = 1"
data = service.search(query).to_table().to_pandas()
print(len(data))

In [ ]:
data['rmi'] = data['mag_r_cModel'] - data['mag_i_cModel']

In [ ]:
tx = np.where(data['mag_i_cModel'][:] < 25)[0]
print(len(tx))

In [ ]:
points = hv.Points((data['rmi'][tx], data['mag_i_cModel'][tx]))

In [ ]:
boundsxy = (0, 0, 0, 0)
box = streams.BoundsXY(source=points, bounds=boundsxy)
bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

In [ ]:
p = dynspread(datashade(points, cmap="hot"))
p = p.opts(width=500, height=500, padding=0.05, show_grid=False, bgcolor='black',
           xlim=(-2.0, 2.0), ylim=(25, 14), xlabel="(r-i) color", ylabel="i-band mag")
p * bounds

If the plot above flashes a flame and then goes all black, just interact with it 
a bit with your mouse (e.g., slightly zoom out) and usually it reappears.

Not sure what the bug is there, bonus points if someone can make this plot sturdier!!

Here's a screenshot so you can see what I made, just in case:

<img src="figures/solution4_figure1.png" width="400px">